In [1]:
%%bash
# Downloaded from https://github.com/KaimingHe/deep-residual-networks
#cd /home/iliauk/.chainer/dataset/pfnet/chainer/models/
#wget https://ikpublictutorial.blob.core.windows.net/deeplearningframeworks/ResNet-50-model.caffemodel

In [2]:
import os
import sys
import numpy as np
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import optimizers
from chainer import cuda
from common.params_inf import *
from common.utils import *

In [3]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.14.1
GPU:  ['Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB']
CUDA Version 8.0.61
CuDNN Version  6.0.21


In [5]:
# Create batches of fake data
fake_input_data_cl, fake_input_data_cf = give_fake_data(BATCH_SIZE*BATCHES_GPU)
print(fake_input_data_cl.shape, fake_input_data_cf.shape)

(1280, 224, 224, 3) (1280, 3, 224, 224)


In [6]:
resnet50 = chainer.links.ResNet50Layers(pretrained_model="auto")
# GPU
chainer.cuda.get_device(0).use()  # Make a specified GPU current
resnet50.to_gpu()  # Copy the model to the GPU

In [7]:
resnet50.available_layers

['conv1', 'pool1', 'res2', 'res3', 'res4', 'res5', 'pool5', 'fc6', 'prob']

In [8]:
def predict_fn(classifier, data, batchsize):
    """ Return features from classifier """
    out = np.zeros((len(data), RESNET_FEATURES), np.float32)
    with chainer.using_config('train', False), chainer.using_config('enable_backprop', False):  
        for idx, dta in yield_mb_X(data, batchsize):
            pred = classifier(cuda.to_gpu(dta), layers=['pool5'])
            out[idx*batchsize:(idx+1)*batchsize] = cuda.to_cpu(pred['pool5'].data).squeeze()        
    return out

In [9]:
cold_start = predict_fn(resnet50, fake_input_data_cf, BATCH_SIZE)

In [10]:
%%time
features = predict_fn(resnet50, fake_input_data_cf, BATCH_SIZE)

CPU times: user 3.52 s, sys: 7.74 ms, total: 3.53 s
Wall time: 3.52 s


In [11]:
print("Images per second {}".format((BATCH_SIZE*BATCHES_GPU)/3.52))

Images per second 363.6363636363636
